In [23]:
"""!conda install -c conda-forge openexr-python -y
!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir 
!pip install --upgrade google-cloud-storage"""

'!conda install -c conda-forge openexr-python -y\n!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir \n!pip install --upgrade google-cloud-storage'

In [49]:
# Imports
import os
import tarfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm
import pickle

from waymo_open_dataset.wdl_limited.sim_agents_metrics import metric_features
from waymo_open_dataset.wdl_limited.sim_agents_metrics import metrics

from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import sim_agents_metrics_pb2
from waymo_open_dataset.protos import sim_agents_submission_pb2
from google.protobuf import text_format

from waymo_open_dataset.utils.sim_agents import submission_specs
from waymo_open_dataset.utils.sim_agents import test_utils as sim_agents_test_utils
from waymo_open_dataset.utils.sim_agents import visualizations
from waymo_open_dataset.utils import trajectory_utils

# Set matplotlib to jshtml so animations work with colab.
from matplotlib import rc
rc('animation', html='jshtml')

# deactivating the warnings
import warnings
warnings.filterwarnings('ignore')

from download import download_from_gcs
from prepare_dataset import Prepare_train_dataset, Prepare_validation_dataset, Prepare_test_dataset
from data_preprocess import process_waymo_data_with_scenario_proto
from Mydataloader import create_dataloader


# Downloader

In [25]:
# Download samples
download_from_gcs('uncompressed/scenario/validation/validation.tfrecord-00000-of-00150')
download_from_gcs('uncompressed/scenario/training/training.tfrecord-00000-of-01000')
download_from_gcs('uncompressed/scenario/testing/testing.tfrecord-00000-of-00150')

validation.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/validation/validation.tfrecord-00000-of-00150
training.tfrecord-00000-of-01000 already exists in waymo_open_dataset_/training/training.tfrecord-00000-of-01000
testing.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/testing/testing.tfrecord-00000-of-00150


# Preprocess

In [26]:
!python data_preprocess.py

2023-05-28 12:16:55.288877: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 12:16:55.588421: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-28 12:16:55.588462: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-28 12:16:59.106980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

## Trajectory data 

In [31]:
# load scenarios/processed_scenarios_training/sample_1a8cc570d620bd31.pkl
pickle_file = 'scenarios/processed_scenarios_training/sample_1a8cc570d620bd31.pkl'
processed = pickle.load(open(pickle_file, 'rb'))

print('The shape of the original trajectories is: ', processed['track_infos']['trajs'].shape)
print('The shape of the tracks to predict until current time step is: ', processed['track_infos']['tracks_to_predict_until_current'].shape)
print('The shape of the tracks to predict after current time step is: ', processed['track_infos']['tracks_to_predict_future'].shape)
print('The shape of the track of adv until current time step is: ', processed['track_infos']['track_of_adv_until_current'].shape)
print('The shape of the track of adv after current time step is: ', processed['track_infos']['track_of_adv_future'].shape)
print('The shape of the track of other agents until current time step is: ', processed['track_infos']['tracks_of_other_agents_until_current'].shape)
print('The shape of the track of other agents after current time step is: ', processed['track_infos']['tracks_of_other_agents_future'].shape)

The shape of the original trajectories is:  (105, 91, 10)
The shape of the tracks to predict until current time step is:  (4, 11, 10)
The shape of the tracks to predict after current time step is:  (4, 80, 10)
The shape of the track of adv until current time step is:  (11, 10)
The shape of the track of adv after current time step is:  (80, 10)
The shape of the track of other agents until current time step is:  (104, 11, 10)
The shape of the track of other agents after current time step is:  (104, 80, 10)


## Map data

In [39]:
for key in processed['map_infos'].keys():
    print('The (shape, type) of the map info {} is: '.format(key), (np.array(processed['map_infos'][key]).shape, type(processed['map_infos'][key])))
    if key != 'all_polylines':
        print('The map info inside each {} element are: '.format(key), processed['map_infos'][key][0].keys())

The (shape, type) of the map info lane is:  ((751,), <class 'list'>)
The map info inside each lane element are:  dict_keys(['id', 'speed_limit_mph', 'type', 'interpolating', 'entry_lanes', 'exit_lanes', 'left_boundary', 'right_boundary', 'polyline_index'])
The (shape, type) of the map info road_line is:  ((77,), <class 'list'>)
The map info inside each road_line element are:  dict_keys(['id', 'type', 'polyline_index'])
The (shape, type) of the map info road_edge is:  ((79,), <class 'list'>)
The map info inside each road_edge element are:  dict_keys(['id', 'type', 'polyline_index'])
The (shape, type) of the map info stop_sign is:  ((1,), <class 'list'>)
The map info inside each stop_sign element are:  dict_keys(['id', 'lane_ids', 'position', 'polyline_index'])
The (shape, type) of the map info crosswalk is:  ((12,), <class 'list'>)
The map info inside each crosswalk element are:  dict_keys(['id', 'polyline_index'])
The (shape, type) of the map info speed_bump is:  ((18,), <class 'list'>

# Data loader

In [52]:
# Example usage
train_directory = 'scenarios/processed_scenarios_training'
validation_directory = 'scenarios/processed_scenarios_validation'
batch_size = 4
train_dataloader = create_dataloader(train_directory, batch_size)
validation_dataloader = create_dataloader(validation_directory, batch_size)


# Iterate over the dataloader
for i, batch in enumerate(train_dataloader):
    print('----------------- batch {} -----------------'.format(i))
    for j, item in enumerate(batch):
        trajectories, tracks_until_current, tracks_future, adv_until_current, adv_future, other_agents_until_current, other_agents_future = item
        print('----------------- item {} -----------------'.format(j))
        print('The shape of the original trajectories is:', trajectories.shape)
        print('The shape of the tracks to predict until current time step is:', tracks_until_current.shape)
        print('The shape of the tracks to predict after current time step is:', tracks_future.shape)
        print('The shape of the track of adv until current time step is:', adv_until_current.shape)
        print('The shape of the track of adv after current time step is:', adv_future.shape)
        print('The shape of the track of other agents until current time step is:', other_agents_until_current.shape)
        print('The shape of the track of other agents after current time step is:', other_agents_future.shape)
        print()
    if i > 3:
        break  


----------------- batch 0 -----------------
----------------- item 0 -----------------
The shape of the original trajectories is: torch.Size([19, 91, 10])
The shape of the tracks to predict until current time step is: torch.Size([2, 11, 10])
The shape of the tracks to predict after current time step is: torch.Size([2, 80, 10])
The shape of the track of adv until current time step is: torch.Size([11, 10])
The shape of the track of adv after current time step is: torch.Size([80, 10])
The shape of the track of other agents until current time step is: torch.Size([18, 11, 10])
The shape of the track of other agents after current time step is: torch.Size([18, 80, 10])

----------------- item 1 -----------------
The shape of the original trajectories is: torch.Size([377, 91, 10])
The shape of the tracks to predict until current time step is: torch.Size([4, 11, 10])
The shape of the tracks to predict after current time step is: torch.Size([4, 80, 10])
The shape of the track of adv until curren